In [1]:
import cvxpy as cp
import numpy as np

K = 100
R = 100
L = 1
verbose = 0

## Define Ak sequence and theta matrix
b = np.ones(K) * 0.01
lmbda = np.array([0.4, 0.5, 0.2, 0.5, 0.3, 0.4, 0.9, 0.9])
lmbda[-1] = 0.9

a = np.zeros(K+1)
a[0] = 1
A = np.zeros(K+1)
A[0] = 1

for i in range(K):
    a[i+1] = (i+1+4)/4
    A[i+1] = A[i] + a[i+1]

theta = np.zeros((K, K))
for k in range(K):
    for i in range(K):
        if i == k:
            theta[k, i] = 1 + (2 * a[k] - 1) * a[k+1] / A[k+1]
        else:
            if i > k:
                break
            theta[k, i] = (a[k+1] / A[k+1]) * (2 * a[i] - theta[k-1, i]) + theta[k-1, i]


In [2]:
# use cvxpy to solve the problem
tolerance = 1e-8
dimG = 2 * K + 2
dimF = K + 1

# define unit vectors
f = np.matrix(np.eye(K + 1))
fs = np.zeros((K + 1, 1))

xs = np.zeros((2 * K + 2, 1))

e = np.matrix(np.zeros((2 * K + 2, K)))
e[1 : K + 1, :] = np.eye(K)
g = np.matrix(np.zeros((2 * K + 2, K + 1)))
g[K + 1 :, :] = np.eye(K + 1)
gs = np.zeros((2 * K + 2, 1))

# define  xi matrix (each column represents a xi vector)
xi = np.matrix(np.zeros((2 * K + 2, K + 1)))
xi[0] = 1

for k in range(K):
    temp = xi[:, 0]
    for i in range(k + 1):
        temp = temp - 1 / L * theta[k ,i] * (e[:, i] + g[:, i])
    xi[:, k + 1] = temp



v_i = cp.Variable((K), nonneg=True)
v_si = cp.Variable(K+1,nonneg=True)
tau = cp.Variable(1, nonneg=True)
u = cp.Variable(K, nonneg=True)

constraints = [v_i == A[0:K]/A[K], v_si >= 0, tau == 1/A[K]/4, u >= 0]

# linear constraints
func_linear = f[:, -1] + v_si[K] * (fs - f[:, K])
for i in range(K):
    func_linear = func_linear + v_i[i] * (f[:, i] - f[:, i + 1]) + v_si[i] * (fs - f[:, i])

func_PSD = tau * xi[:, 0] @ xi[:, 0].T + 1 / (2 * L) * g[:, K] @ g[:, K].T

for i in range(K):
    Aij = (
        1/2
        * (
            (xi[:, i] - xi[:, i + 1]) @ g[:, i + 1].T
            + g[:, i + 1] @ (xi[:, i] - xi[:, i + 1]).T
        )
        + 1 / (2 * L) * (g[:, i] - g[:, i + 1]) @ (g[:, i] - g[:, i + 1]).T
    )
    Asi = (
        1/2 * ((xs - xi[:, i]) @ g[:, i].T + g[:, i] @ (xs - xi[:, i]).T)
        + 1 / (2 * L) * (gs - g[:, i]) @ (gs - g[:, i]).T
    )
    func_PSD = func_PSD + v_i[i] * Aij + v_si[i] * Asi + u[i] * e[:, i] @ e[:, i].T

i = K
Asi = (
    1/2 * ((xs - xi[:, i]) @ g[:, i].T + g[:, i] @ (xs - xi[:, i]).T)
    + 1 / (2 * L) * (gs - g[:, i]) @ (gs - g[:, i]).T
)
func_PSD = func_PSD + v_si[i] * Asi

constraints += [func_linear == 0]
constraints += [func_PSD >> 0]
obj = cp.Minimize(tau * R**2 + b @ u)

solver_opt = {'solver': 'MOSEK', 'verbose': verbose, 'mosek_params': {'MSK_DPAR_INTPNT_CO_TOL_PFEAS': tolerance}}
prob = cp.Problem(obj, constraints)
prob.solve(**solver_opt)

# print status
print("status:", prob.status)
print("u=",u.value)

status: optimal
u= [ 34.39659999  43.10116454  51.73245583  60.33803037  68.91427118
  77.51749839  86.10015041  94.68683266 103.27959699 111.8378111
 120.37638885 128.9028962  137.4042039  145.90141461 154.38006673
 162.81844903 171.23776134 179.62549292 187.95907796 196.249727
 204.49831722 212.69800718 220.8438454  228.93101178 236.95542605
 244.91245693 252.79631522 260.6013029  268.31731458 275.94127885
 283.47523835 290.90811047 298.23130443 305.43892715 312.52553352
 319.48564042 326.31304583 332.99958471 339.53626088 345.91510531
 352.12901217 358.17070126 364.03242471 369.70614899 375.1835679
 380.45613252 385.51515768 390.35171616 394.95660966 399.32039448
 403.4334444  407.28600516 410.86820904 414.1700521  417.18135871
 419.89175501 422.29065608 424.36728237 426.11066314 427.50964565
 428.55289978 429.22891936 429.52602611 429.43236014 428.93589013
 428.02440291 426.68550727 424.90662787 422.6750001  419.97766524
 416.80146752 413.13305371 408.95887574 404.26519502 399.0380

In [3]:
A_seq = np.append(A,0)
u_theoretical = [0 for i in range(K)]

for i in range(K):
    u_temp = A_seq[i]*(1+2*(A_seq[i+1]-A_seq[i]))* (A_seq[i]+2*(A_seq[i]-A_seq[i-1])*(A_seq[i+1]-A_seq[i]))/(4*L * A_seq[K]*(A_seq[i+1]-(A_seq[i+1]-A_seq[i])**2))
    for k in range(i+1,K):
            u_temp = u_temp + A_seq[k]*(A_seq[i]-A_seq[i-1])*(A_seq[k+1]-A_seq[k])*(1+2*(A_seq[k+1]-A_seq[k])) / (2*L * A_seq[K]*(A_seq[k+1]-(A_seq[k+1]-A_seq[k])**2))
    
    u_theoretical[i] = u_temp

u_theoretical - u.value

array([ 6.80815906e-02, -2.19210653e-02, -4.05243622e-02, -3.61818302e-02,
       -6.35192657e-03, -8.60985737e-03,  3.17037180e-03,  2.76661685e-03,
       -1.36712929e-02, -7.49132053e-03,  4.23023231e-03,  1.15825657e-02,
        2.51675381e-02,  2.11724270e-02,  1.11659789e-02,  1.37837341e-02,
        4.56722659e-03, -7.41442249e-03, -3.22003634e-03,  2.13214654e-03,
        3.77411685e-03,  4.37339004e-03,  4.52435041e-03,  4.50683349e-03,
        3.67763984e-03,  1.76138860e-03, -5.42602775e-04, -2.80944402e-03,
       -3.90111892e-04,  4.14718797e-03,  2.93692270e-03,  1.05555368e-03,
        9.04442097e-04,  2.00393363e-03,  3.24315900e-03,  3.36594535e-03,
        1.65171243e-03, -8.40374974e-04, -2.40334429e-03, -2.54065010e-03,
       -1.80227530e-03, -7.47153118e-04,  3.50155167e-04,  1.31742013e-03,
        2.07176821e-03,  2.58972190e-03,  2.80072259e-03,  2.69294227e-03,
        2.34231489e-03,  1.88683507e-03,  1.46368633e-03,  1.15477119e-03,
        9.72123048e-04,  